In [1]:
from pathlib import Path
from dataclasses import dataclass
import os
os.chdir("../")

In [8]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    subsample: float
    scale_pos_weight: float
    n_estimators: int
    min_child_weight:int
    max_depth: int
    learning_rate: float
    gamma: float
    colsample_bytree: float
    target_column: str

In [9]:
from src.mlpro.constants import *
from src.mlpro.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.xgboost_param
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            subsample= params.subsample,
            scale_pos_weight = params.scale_pos_weight,
            n_estimators= params.n_estimators,
            min_child_weight=params.min_child_weight,
            max_depth=params.max_depth,
            learning_rate=params.learning_rate,
            gamma=params.gamma,
            colsample_bytree=params.colsample_bytree,
            target_column=schema.name
        )

        return model_trainer_config

In [11]:
from xgboost import XGBClassifier
import joblib
import pandas as pd
from src.mlpro import logger

In [14]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1).to_numpy()
        train_y = train_data[self.config.target_column].values


        xgb = XGBClassifier(
        subsample=self.config.subsample,
        scale_pos_weight=self.config.scale_pos_weight,
        n_estimators=self.config.n_estimators,
        min_child_weight=self.config.min_child_weight,
        max_depth=self.config.max_depth,
        learning_rate=self.config.learning_rate,
        gamma=self.config.gamma,
        colsample_bytree=self.config.colsample_bytree,
        random_state=108)


        xgb.fit(train_x, train_y)

        joblib.dump(xgb, os.path.join(
            self.config.root_dir, self.config.model_name))

In [15]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-10-13 17:46:43,013: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-10-13 17:46:43,016: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-13 17:46:43,019: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-10-13 17:46:43,020: INFO: common: created directory at: artifacts]
[2024-10-13 17:46:43,021: INFO: common: created directory at: artifacts/model_trainer]
